In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [2]:
nQuizz=10
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [4]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'APPADOO et VICENTINI': ['412653', '415269'],
 'Aaron Dieu': ['413093'],
 'Abel': ['417920'],
 'Abigail raguraj': ['397598'],
 'Afonso Peleja': ['415773'],
 'Agathe Corbier': ['398357'],
 'Agleizes001': ['412341'],
 'Aiden Bertin': ['415098'],
 'Alexis peyen': ['414807'],
 'Alice': ['400421'],
 'Alice Dupouy': ['400421'],
 'Alicia PARISET': ['397359'],
 'Alicia PARISET Axel Hennebert': ['397359', '414202'],
 u'Aliz\xe9e Artaud': ['415398'],
 'Amblard Juline': ['415539'],
 'Ambre Sovieri': ['402017'],
 'Amelie Martin': ['398003'],
 'Amine Elhaloui': ['415818'],
 'Amine elhaloui': ['415818'],
 u'Am\xe9lie Martin': ['398003'],
 'Anais Bernou': ['413687'],
 u'Ana\xefs AURIAULT': ['418584'],
 u'Ana\xefs Bernou': ['413687'],
 'Angele': ['417705'],
 u'Anna\xeblle Gatien': ['417456'],
 'Anne-Emmanuelle': ['421063'],
 'Appadoo': ['412653'],
 'Appadoo et Vicentini': ['415269', '412653'],
 'Archi': ['397427'],
 'Archibald Schmidt': ['397427'],
 u'Artaud Aliz\xe9e': ['415398'],
 u'Astrid Barri\xe

In [5]:
len(alias)

518

In [6]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(filepath_or_buffer=fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [7]:
# lQuizz, dfCM

# Listage des alias

In [8]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM

,id,nom,prénom,email,ine,PN,NP
0,416792,ABDEL GAWWAD,HANIA,hania.abdel-gawwad@etu.u-bordeaux-montaigne.fr,080761564FE,hania abdel gawwad,abdel gawwad hania
1,415014,ADROVER,KILLIAN,killian.adrover@etu.u-bordeaux-montaigne.fr,081310016EA,killian adrover,adrover killian
2,417623,AGUILAR,JULIETTE,juliette.aguilar@etu.u-bordeaux-montaigne.fr,061226840EF,juliette aguilar,aguilar juliette
3,421315,AKARIDE,IKRAM,ikram.akaride@etu.u-bordeaux-montaigne.fr,0756AU00FD1,ikram akaride,akaride ikram
4,416758,ALBANEL,FANNY,fanny.albanel@etu.u-bordeaux-montaigne.fr,090927602AE,fanny albanel,albanel fanny
5,414118,ALBRESPY,THEO,theo.albrespy@etu.u-bordeaux-montaigne.fr,061057341JF,theo albrespy,albrespy theo
6,420573,ALESSANDRI,ORNELLA,ornella.alessandri@etu.u-bordeaux-montaigne.fr,213042901EC,ornella alessandri,alessandri ornella
7,397859,ALPHONSINE,JUSTINE,justine.alphonsine@etu.u-bordeaux-montaigne.fr,091152973GB,justine alphonsine,alphonsine justine
8,418829,ALVAREZ,CAMILLE,camille.alvarez@etu.u-bordeaux-montaigne.fr,072082252KH,camille alvarez,alvarez camille
9,415539,AMBLARD,JULINE,juline.amblard@etu.u-bordeaux-montaigne.fr,060012103FB,juline amblard,amblard juline


In [9]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Théo
Marie
Minou minou
Tintin
Justine A
XuehangWang
Archiboule le plus fort
Mina
lulu
Patoox
Romane
camilleS
minou
Sasa
Sali


In [10]:
for nP,pseudo in enumerate(pseudos):
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudo]=idPseudo

Théo
Nom a chercher:

Marie
Nom a chercher:

Minou minou
Nom a chercher:

Tintin
Nom a chercher:

Justine A
Nom a chercher:

XuehangWang
Nom a chercher:

Archiboule le plus fort
Nom a chercher:

Mina
Nom a chercher:

lulu
Nom a chercher:

Patoox
Nom a chercher:

Romane
Nom a chercher:

camilleS
Nom a chercher:

minou
Nom a chercher:

Sasa
Nom a chercher:

Sali
Nom a chercher:



In [11]:
alias["Lilly Boutelet 1"]

['387089']

In [12]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [13]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

             PN      id
56  emma bismes  413429


In [14]:
len(pseudos)

15

In [15]:
# alias["Sofiane"]

In [16]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

In [17]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]



In [18]:
for al in alias:
    if not isinstance(alias[al],list):
        alias[al]=[alias[al]]

In [19]:
len(alias)

518

In [20]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'APPADOO et VICENTINI': ['412653', '415269'],
  'Aaron Dieu': ['413093'],
  'Abel': ['417920'],
  'Abigail raguraj': ['397598'],
  'Afonso Peleja': ['415773'],
  'Agathe Corbier': ['398357'],
  'Agleizes001': ['412341'],
  'Aiden Bertin': ['415098'],
  'Alexis peyen': ['414807'],
  'Alice': ['400421'],
  'Alice Dupouy': ['400421'],
  'Alicia PARISET': ['397359'],
  'Alicia PARISET Axel Hennebert': ['397359', '414202'],
  u'Aliz\xe9e Artaud': ['415398'],
  'Amblard Juline': ['415539'],
  'Ambre Sovieri': ['402017'],
  'Amelie Martin': ['398003'],
  'Amine Elhaloui': ['415818'],
  'Amine elhaloui': ['415818'],
  u'Am\xe9lie Martin': ['398003'],
  'Anais Bernou': ['413687'],
  u'Ana\xefs AURIAULT': ['418584'],
  u'Ana\xefs Bernou': ['413687'],
  'Angele': ['417705'],
  u'Anna\xeblle Gatien': ['417456'],
  'Anne-Emmanuelle': ['421063'],
  'Appadoo': ['412653'],
  'Appadoo et Vicentini': ['415269', '412653'],
  'Archi': ['397427'],
  'Archibald Schmidt': ['397427'],
  u'Artau

In [21]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [22]:
nbQ=6.0

In [23]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [24]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,laïs tejerizo,5,104.53,[401188],18,401188,None,None
1,csaintavi001 ebismes001,5,221.52,"[415962, 413429]",18,415962,413429,None
2,Emma Luzinski,5,245.84,[413297],18,413297,None,None
3,Chloé Gravelat,5,302.20,[413916],18,413916,None,None
4,FLEURANCEAU Sarah,5,313.92,[418543],18,418543,None,None
5,L.Champernau,4,133.98,[323999],17,323999,None,None
6,Meyrignac-Platon Clélia,4,175.02,[408800],17,408800,None,None
7,Léa Hardouin,4,189.54,[413012],17,413012,None,None
8,Thaïs Ferrand,4,201.02,[415136],17,415136,None,None
9,Simon SAINTARAILLES,4,223.39,[413050],17,413050,None,None


In [64]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [65]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True])
df123

,pseudo,note,temps,id
0,laïs tejerizo,18,104.53,401188
1,csaintavi001 ebismes001,18,221.52,415962
1,csaintavi001 ebismes001,18,221.52,413429
2,Emma Luzinski,18,245.84,413297
3,Chloé Gravelat,18,302.20,413916
4,FLEURANCEAU Sarah,18,313.92,418543
5,L.Champernau,17,133.98,323999
6,Meyrignac-Platon Clélia,17,175.02,408800
7,Léa Hardouin,17,189.54,413012
8,Thaïs Ferrand,17,201.02,415136


In [66]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")